## Creating a List of PDF transcripts present on the Hard Drive

In [1]:
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
import os

rootdir = '../Industrials'
techrootdir = '../InformationTechnology' # have not use it yet
fileList = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith(".pdf"):
            fileList.append(filepath)
for subdir, dirs, files in os.walk(techrootdir):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith(".pdf"):
            fileList.append(filepath)

## Extracting Text from PDF

In [2]:
#https://www.geeksforgeeks.org/working-with-pdf-files-in-python/
import PyPDF2

text = []

for file in fileList:
    
    pdfFileObj = open(file, 'rb')

    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    #pageObj = pdfReader.getPage(0)
    
    for page in pdfReader.pages:
        text = text + page.extractText().split('\n')


In [3]:
# text

## Creating Bag of Words from Text extracted from PDFs

In [4]:
#https://github.com/eclarson/MachineLearningNotebooks/blob/master/02_Document%20Feature%20Engineering.ipynb
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english')
# count_vect = CountVectorizer()

bag_words = count_vect.fit_transform(text)

In [5]:
# print(len(count_vect.vocabulary_))
# print(count_vect.vocabulary_)

In [6]:
import pandas as pd

pd.options.display.max_columns = 999
df = pd.DataFrame(data=bag_words.toarray(),columns=count_vect.get_feature_names())

## List of 50 words that appear most often

In [7]:
# df.sum().sort_values()[-50:]

## Filtering out most commonly used words from Bag of Words

In [8]:
commonlist = ['the', 'and', 'to', 'in', 'of', 'that', 'on', 'is', 'for', 'it', 'as', 'this', 're', 'are', 'be', 
              'at', 'from', 'have', 'about', 'by', 'just', 'or', 'there', 'what', 'were', 'will', 'not']

In [9]:
for i in range(len(text)):
    text[i] = ' '.join([i for i in text[i].split(' ') if i not in commonlist])

In [10]:
bag_words = count_vect.fit_transform(text)
df = pd.DataFrame(data=bag_words.toarray(),columns=count_vect.get_feature_names())

## Filtered Words

In [11]:
# df.sum().sort_values()[-50:]

## Pick top and bottom 15% frequent words

In [12]:
fwordlist = df.sum().sort_values()
rownum15 = (int)(fwordlist.shape[0]*0.15)
terms = [fwordlist[:rownum15], fwordlist[-rownum15:]]
terms = pd.concat(terms)
terms = terms.to_frame()
word_indexes = terms.index
# word_indexes

In [13]:
terms[4300:4388]

,0
continued,1302
points,1307
comes,1314
don,1316
rate,1322
capital,1331
point,1349
know,1369
markets,1371
margins,1395


## Fetch ticker and date from PDF

In [14]:
# fetch ticker and date from calls
import tika
from tika import parser

tickerdate_list = []

for file in fileList:
    PDF_Parse = parser.from_file(file)
    trimmed = PDF_Parse ['content'].strip().split("\n")
    tempTicker = trimmed[5].split(":")
    tempDate = trimmed[7].split(":")
    ftype = -1
    if file.split("/")[1] == "Industrials":
        ftype = 1
    elif file.split("/")[1] == "InformationTechnology":
        ftype = 2

    tickerdatedic = {
        "file": PDF_Parse ['content'].strip(),
        "type": ftype,
        tempTicker[0].strip(): tempTicker[1].strip().split(" ")[0],
        tempDate[0].strip(): tempDate[1].strip()
    }
    tickerdate_list.append(tickerdatedic)
# print(tickerdate_list[0])

2019-05-04 14:46:51,562 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


## Decide a good or bad call and scoring a word
### Running takes about 2.5mins
### showTest = True/False: includes date, open/close price, diff-price

In [15]:
# get close and open price by ticker name
from pandas_datareader import data
import numpy
import datetime
import time
start = time.time()

word_scores = {}
for word in word_indexes:
    word_scores[word] = 0
updateIter = 100
updateCount = [0,0,0,0,0,0,0,0,0,0]
scoreMatrix = numpy.zeros(shape=(100,10))
acc = 100
    
for dic in tickerdate_list:
    sign = 0 # @sign: good or bad
    year = int(dic.get("Date").split("-")[0]) # @year: year
    showTest = False # -------------------@showTest: date, open/close prise, diff-price-----------------------
    
    # pre-hour or post-hour
    if dic.get("type") == 1:
        date = datetime.datetime.strptime(dic.get("Date"), '%Y-%m-%d')
        date = date+datetime.timedelta(days=-1)
        date = datetime.datetime.strftime(date, '%Y-%m-%d')
        panel_data = data.DataReader(dic.get("Company Ticker"), 'yahoo', date, date)
        if showTest:
            print(date) # test
            print(panel_data) # test
    elif dic.get("type"):
        date = dic.get("Date")
        panel_data = data.DataReader(dic.get("Company Ticker"), 'yahoo', date, date)
    # get good or bad
    try:
      diff = panel_data.iloc[1]["Close"]-panel_data.iloc[0]["Close"]
    except IndexError:
      continue
    if showTest:
        print(diff) # test
    if diff >= 0:
        sign = 1
    else:
        sign = -1
    
    # update scores
    currentYear = 2019
    yearDiff = currentYear-year+1
    bonus_base = 2 # score weight base
    dy_bonus = sign * (bonus_base ** (1.0/yearDiff))
    for word in word_indexes:
        if word in dic.get("file"):
            preScore = word_scores[word]
            if word_scores.get(word) == 0:
                word_scores[word] = dy_bonus
            else:
                word_scores[word] = word_scores.get(word) + dy_bonus
                # Words to use: good, think, growth, business, sales, expect, strong, really, question, new
            if (word == "good")&(updateCount[0]<updateIter):
                scoreMatrix[updateCount[0], 0] = round(word_scores[word],4)/acc
                updateCount[0]+=1
            elif (word == "think")&(updateCount[1]<updateIter):
                scoreMatrix[updateCount[1], 1] = round(word_scores[word],4)/acc
                updateCount[1]+=1
            elif (word == "growth")&(updateCount[2]<updateIter):
                scoreMatrix[updateCount[2], 2] = round(word_scores[word],4)/acc
                updateCount[2]+=1
            elif (word == "costs")&(updateCount[3]<updateIter):
                scoreMatrix[updateCount[3], 3] = round(word_scores[word],4)/acc
                updateCount[3]+=1
            elif (word == "sales")&(updateCount[4]<updateIter):
                scoreMatrix[updateCount[4], 4] = round(word_scores[word],4)/acc
                updateCount[4]+=1
            elif (word == "expect")&(updateCount[5]<updateIter):
                scoreMatrix[updateCount[5], 5] = round(word_scores[word],4)/acc
                updateCount[5]+=1
            elif (word == "strong")&(updateCount[6]<updateIter):
                scoreMatrix[updateCount[6], 6] = round(word_scores[word],4)/acc
                updateCount[6]+=1
            elif (word == "okay")&(updateCount[7]<updateIter):
                scoreMatrix[updateCount[7], 7] = round(word_scores[word],4)/acc
                updateCount[7]+=1
            elif (word == "oil")&(updateCount[8]<updateIter):
                scoreMatrix[updateCount[8], 8] = round(word_scores[word],4)/acc
                updateCount[8]+=1
            elif (word == "ago")&(updateCount[9]<updateIter):
                scoreMatrix[updateCount[9], 9] = round(word_scores[word],4)/acc
#                 print(scoreMatrix[updateCount[9], 9])
                updateCount[9]+=1
# end = time.time()
# print(end - start)
# print(word_scores)


## Dynamic visualization of scores

In [20]:
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib
from IPython import display

%matplotlib qt5

fig=plt.figure(figsize=(10, 6))

n=10 #Number of frames
x=["good", "think", "growth", "costs", "sales", "expect", "strong", "okay", "oil", "ago"]
data = scoreMatrix[40:50]
# print(data)
barcollection = plt.bar(x,data[0])
axes = plt.gca()
axes.set_ylim([-0.05,0.3])

def animate(i):
    y=data[i]
    for i, b in enumerate(barcollection):
        b.set_height(y[i])

anim=animation.FuncAnimation(fig,animate,repeat=False,blit=False,frames=n,
                             interval=5000)

plt.show()
display.clear_output(wait=True)


## Audio to text

In [ ]:
# from pydub import AudioSegment
# import speech_recognition as sr
# import os

# rootdir = './'
# fileList = []
# for subdir, dirs, files in os.walk(rootdir):
#     for file in files:
#         #print os.path.join(subdir, file)
#         filepath = subdir + os.sep + file

#         if filepath.endswith(".mp3"):
#             fileList.append(filepath)

# AudioSegment.converter = "./ffmpeg"
# r = sr.Recognizer()
# alltext = ""
# for file in fileList:
#     sound = AudioSegment.from_mp3(file)
#     segments = []
#     start = 0
#     stop = 6000
#     while(stop<len(sound)):
#         segments.append(sound[start:stop])
#         start=stop
#         stop+=6000
#     if start<len(sound):
#         segments.append(sound[start:len(sound)])
        
#     for segment in segments:
#         segment.export("temp.wav", format="wav")
#         hav = sr.AudioFile("temp.wav")
#         with hav as source:
#            audio = r.record(source)
#         try:
#             text = r.recognize_google(audio)
#             alltext += text
# #             print(text+" ")
#         except:
#           continue
# #     print(alltext)
#     fo = open(file.split("//")[1].split(".")[0]+".txt", "w")
#     fo.write(alltext);
#     fo.close()
#     break

## Scoring Calls

In [ ]:
# import os
# from sklearn.feature_extraction.text import CountVectorizer
# import pandas as pd

# rootdir = './'
# fileList = []
# scoreList = []
# for subdir, dirs, files in os.walk(rootdir):
#     for file in files:
#         #print os.path.join(subdir, file)
#         filepath = subdir + os.sep + file

#         if filepath.endswith(".txt"):
#             fileList.append(filepath)

# for txt in fileList:
#     fileObj = open(txt, "r")
#     text = fileObj.read().split("\n")
#     fileObj.close()
#     count_vect = CountVectorizer(stop_words='english')
#     commonlist = ['the', 'and', 'to', 'in', 'of', 'that', 'on', 'is', 'for', 'it', 'as', 'this', 're', 'are', 'be', 
#                   'at', 'from', 'have', 'about', 'by', 'just', 'or', 'there', 'what', 'were', 'will', 'not']
#     for i in range(len(text)):
#         text[i] = ' '.join([i for i in text[i].split(' ') if i not in commonlist])
#     bag_words = count_vect.fit_transform(text)
#     df = pd.DataFrame(data=bag_words.toarray(),columns=count_vect.get_feature_names())

#     fwordlist = df.sum().sort_values()
#     word_indexes = fwordlist.index
# #     print(word_indexes)
#     totalscore = 0
#     for word in word_indexes:
#         if word_scores.get(word)==None:
#             continue
#         totalscore += word_scores.get(word)
#     scoreList.append(totalscore)
# print(scoreList)